This notebook answers a few basics questions about the COSMOS field data-- how many objects and candidates were detected, how objects are distributed among candidates, and how those distributions change when an R/B cutoff is applied.

In [ ]:
import psycopg2
import psycopg2.extras
import getpass
from IPython.display import display
import ipywidgets
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
dbuser = input("DB User: ")
dbpasswd = getpass.getpass("DB Password: ")

In [ ]:
db = psycopg2.connect(f"dbname='decat' user='{dbuser}' password='{dbpasswd}' host='decatdb.lbl.gov'")

In [ ]:
db.autocommit = True # Makes the computer nicer about SQL syntax errors

cursor = db.cursor( cursor_factory = psycopg2.extras.DictCursor )

#### How many objects total in the COSMOS field?

In [ ]:
query = ( 'SELECT COUNT(*) FROM objects o '
         'WHERE q3c_radial_query(o.ra,o.dec,150,2.2,3) '
         'LIMIT 10000000')
cursor.execute( query )

objnum = cursor.fetchall()[0][0]
print( objnum )

#### How many good (R/B>0.6) objects total in the COSMOS field?

In [ ]:
query = ( 'SELECT COUNT(*) FROM objects o '
         'WHERE q3c_radial_query(o.ra,o.dec,150,2.2,3) '
         'AND o.rb>0.6 '
         'LIMIT 10000000')
cursor.execute( query )

objnum = cursor.fetchall()[0][0]
print( objnum )

#### How many candidates total in the COSMOS field?

In [ ]:
query = ( 'SELECT COUNT(*) FROM candidates c '
         'WHERE q3c_radial_query(c.ra,c.dec,150,2.2,3) ' 
         'LIMIT 10000000')
cursor.execute( query )

candnum = cursor.fetchall()[0][0]
print( candnum )

#### How many objects per candidate, on average? (COSMOS)

In [ ]:
obj_per_cand_av = objnum / candnum
print( obj_per_cand_av ) # This matches what we get later

del candnum, objnum, obj_per_cand_av

#### How are the objects distributed among candidates (COSMOS, ignoring RB scores)?

In [ ]:
### Returns objects.candidate_id for every object

query = ( 'SELECT candidate_id FROM objects o '
         'WHERE q3c_radial_query(o.ra,o.dec,150,2.2,3) '
         'LIMIT 10000000')
cursor.execute( query )

cndobj_a = np.array( cursor.fetchall() ).transpose()
cndobj_a

In [ ]:
# Finding out how many objects have a given candidate id
candidate_a, frequencies_a = np.unique( cndobj_a[0], return_counts=True )

In [ ]:
pd.DataFrame( frequencies_a, candidate_a, columns=["Number of detections"]) # visualization

In [ ]:
# Plotting it up
plt.figure( figsize=(8,5) )
plt.hist( frequencies_a, bins=100 )
plt.xlim( 0, 170 )
plt.xlabel( "Objects per candidate" )
plt.ylabel( "Number of candidates" )
plt.axvline( np.mean( frequencies_a ), label="Mean objs/cand, %f" % (np.mean(frequencies_a)), color='r' )
plt.legend()
print( "THIS PLOT HAS LIMITED X-AXIS RANGE FOR VISIBILITY, should go out to ~700" )
print( "It also includes every object, regardless of rb score" )

#### Same analysis, but for only "good" object detections (COSMOS, R/B>0.6)

In [ ]:
query = ( 'SELECT candidate_id FROM objects o '
         'WHERE rb > 0.6 '
         'AND q3c_radial_query(o.ra,o.dec,150,2.2,3) ' 
         'LIMIT 10000000')
cursor.execute( query )

cndobj_g = np.array( cursor.fetchall() ).transpose()
cndobj_g

In [ ]:
# Finding out how many objects have a given candidate id
candidate_g, frequencies_g = np.unique( cndobj_g[0], return_counts=True )

In [ ]:
# Plotting it up
plt.figure( figsize=(8,5) )
plt.hist( frequencies_g, bins=150 )
plt.xlim( 0, 50 )
plt.xlabel( "Objects per candidate" )
plt.ylabel( "Number of candidates" )
plt.axvline( np.mean( frequencies_g ), label="Mean objs/cand, %f" % ( np.mean(frequencies_g) ), color='r' )
plt.legend()
print( "THIS PLOT HAS LIMITED X-AXIS RANGE FOR VISIBILITY, should go out to ~350" )
print( "It includes only objects with rb scores above 0.6" )